# 用于知网题录摘要分析的BERTopic

一些说明：
0.知网获取的题录要另存为xlsx格式到题录文件夹，知网的题录表格有些奇怪的毛病，python读取不出来，需要另存为通用的Excel格式，即xlsx格式才能正常读取。
2.提示：bertopic库的安装或许存在一些坑，但是，经测试，从清华源载入的却不易报错。hdbscanze 建议用pycharmd内置的工具安装或手动安装该库，pip似乎并不能直接安装hdbscan。可以尝试安装hdbscan-with-cosine-distance库作为替代。如上述方法无法解决，可运行以下指令(单元去掉注释用的井号#并运行)：
    pip install hdbscan
    conda install -c conda-forge hdbscan
    conda install -c zeus1942 hdbscan
    pip install BERTopic
    如果第一条失败，则运行第二条，第二条失败再运行第三条；如果第一条成功，则直接运行第四条。运行成功则重启jupyter然后将这几行恢复注释，运行后面的代码
2.在使用bertopic前应对模型有所了解，不同的参数配置会产生不一样的效果，一个理想的结果是使用不同参数搭配反复调试之后得到的。代码所提供的是经笔者测试较为通用的搭配，最适合您的语料的参数搭配未必是当前的搭配。
3.bertopic应用于中文的另一种分词方式是在sklearn中构建jieba分词器，但是经测试，效果较差，故采用了现在的预先分词的方式。
4.stop_words_jieba.txt和stop_words_sklearn是两个停用词表，其作用不同，前者用于必要的文本清洗，过度清洗会影响语义模型的正常运作，其停用词需非常谨慎的添加；后者是标准的停用词表，可以将不需要的词语直接写到里面，注意格式为每行一个词语，最后一行不能空行，添加之后保存再重新运行代码。
5.句嵌入模型(embedding model)的选择有很大的自由度，经测试中文或中英混合效果较好的m3e模型：
6.句嵌入模型是整个代码运行中耗时最长的部分，建议具有英伟达显卡的机器运行。若无相应独显，而是在CPU上运行，对超过3000的样本量进行处理时，运行时间可能超过20分钟。


In [ ]:
# pip install hdbscan

conda install -c conda-forge hdbscan 安装hdbscan组件成功率较高

In [ ]:
# conda install -c conda-forge hdbscan

In [ ]:
# conda install -c zeus1942 hdbscan

In [ ]:
# pip install BERTopic

In [ ]:
# pip install jieba

# 数据

In [1]:
import pandas as pd
import os

In [ ]:
# -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
# 将知网下载的xls格式题录（导出）另存为xlsx格式放到题录文件夹中，可以多个文件
# 注意：这一段只执行一次，重复执行会导致重复读取
folder_path = os.getcwd() + '\\' +'题录文件夹'
data0 = pd.DataFrame()
for file_name in os.listdir(folder_path):
    # 确保文件是 Excel 文件
    if file_name.endswith(".xlsx"):
        # 构建完整的文件路径
        file_path = os.path.join(folder_path, file_name)
        # 读取 Excel 文件
        df = pd.read_excel(file_path)
        # 将当前文件的数据合并到总的 DataFrame 中
        data0 = pd.concat([data0, df], ignore_index=True)

# data = data0.head(1000) # 量大的话（比如>3000）先在减量版本上进行测试，如果减量也有好结果，再迁移到更大的样本上
# # CNKI题录量较少的不需要减量，直接用,需要先减量做测试时，用上面的 data = data0.head(1000) 替换下面的 data=data0
data = data0
del data0

In [ ]:
data.head()

In [ ]:
# print(data['Summary-摘要'])

In [ ]:
# 设置所需函数
def read(path):
    f = open(path,encoding="utf8")
    data = []
    for line in f.readlines():
        data.append(line)
    return data

# 加载停用词表
def getword(path):
    swlis = []
    for i in read(path):
        outsw = str(i).replace('\n','')
        swlis.append(outsw)
    return swlis

In [ ]:
# # 构建分词器
import jieba
# import jieba_fast as jieba    # 和import jieba 二选一，但jieba_fast库至少快2-3倍，后续代码无需变动

# 加载用户词表
jieba.load_userdict('关键词表.txt')

# 停用词表
stopLists = list(getword('stop_words_jieba.txt')) # 此处谨慎添加停用词,仅供必要的清洗用,因为这里添加停用词可能会影响文本的语义,干扰后续的模型计算。语义模型不可使用过度清洗的文本！

# 定义分词器
def paperCut(intxt):
    return [w for w in jieba.cut(intxt,cut_all=False,HMM=True) if w not in stopLists and len(w)>=1]
def SentenceCut(doc):
    d = str(doc)
    d = paperCut(d)
    d= " ".join(d)
    return d

In [ ]:
# print(stopLists)

In [ ]:
# 对摘要执行分词
data['Summary-摘要'] = data['Summary-摘要'].apply(SentenceCut)

# **主题模型**

## 模型训练

In [ ]:
# 载入bertopic库
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP

In [ ]:
# 一个经验公式，供参考
import math
n = 0.06 #用于计算主题最低包含文档数量的参数，学术文献建议不高于0.1（因为存在对应专门领域的小主题），评论区采集的文字建议0.2（因为很多同质观点）
p = math.log10(len(data))
q = math.ceil(n*math.pow(p,4))# 一个经验公式，用于控制最小聚类大小。对于学术文献，q大于等于3是合适的，这是因为如果一项研究有较高价值或展开了一个新领域，往往会有同类研究跟进（+1），和原作者自己的续作（+1），即使出现时间较短，有3篇或3篇以上的文献属于该主题也是大概率事件.也可以不用此公式，直接设定q的数值。评论弹幕等，同质性强，建议设置更高的数值。
# q = 3
print(q)


In [ ]:
# 模型路径设置
model_path = os.getcwd() + '\\' +'embedding_models\\moka-ai_m3e-small'

In [ ]:
# 加载sklearn的停用词
stop_words_path = os.getcwd() + '\\' +'stop_words_sklearn.txt'
stop_words_sklearn = list(getword(stop_words_path))#在sklearn处理的阶段再加停用词，这里停用词可以按需添加，不影响句嵌入模型运行。

In [ ]:
# 设定模型参数
model = BERTopic(verbose=True,
                 language="multilingual",
                 embedding_model=model_path,
                 umap_model=UMAP(n_neighbors=64,
                                  n_components=128, # 参考LSA降维的参数设置为128，未必是最优
                                  min_dist=0.00,
                                  metric='cosine',
                                  random_state=100 # 随机种子
                                 ),
                 hdbscan_model=HDBSCAN(min_cluster_size=q,
                                       metric='euclidean',
                                       core_dist_n_jobs=4,
                                       prediction_data=True),
                 vectorizer_model=CountVectorizer(
                                                  stop_words=stop_words_sklearn,# 没有在sklearn中分词，而是预先分词，但停用词需要加在此处
                                                  ngram_range=(1,1),
                                                  binary=False
                                                  ),
                 ctfidf_model=ClassTfidfTransformer(
                                                   bm25_weighting=True,#可选参数，BM25是一种优化的TFIDF类算法，对TFIDF的公式略有修改
                                                   reduce_frequent_words=True),# reduce_frequent_word为True，抑制部分词语，起到软停用的作用，比如可以抑制“可以”“一下”“什么”等词汇。
                 n_gram_range=(1,1), # 是否允许词组，比如 解决 问题 在分词阶段分开，如果参数(1，2)则允许单个词语的同时也允许长度为2的词组，2可以改任意正自然数。
                 calculate_probabilities=True, # 若Ture，以文档归属某模型的概率实现软分类，某种意义上更符合实际，根据需要酌情使用
                 nr_topics="auto", # 减少多少个主题，可以设置为auto，让聚类算法决定是否缩减。学术文件建议不用此参数，即使需要缩减主题，后续可以手动缩减。但如果是初次运行，可以尝试此参数。
                 top_n_words=512, # 低于30，建议10~20，有观点认为10最优，但考虑到停用词，应适当给予余量为宜。若需主题计算余弦相似度等，那就设置很大的数值，如512，1024。
                 min_topic_size=q)


In [ ]:
summary = data['Summary-摘要'].tolist()
headline_topics,prob= model.fit_transform(summary)

## 主题提取与呈现

In [ ]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))

In [ ]:
# freq['Representative_Docs']

In [ ]:
a_topic = freq.iloc[2]["Topic"] # Select one topic
model.get_topic(a_topic) # Show the words and their c-TF-IDF scores

## 所获主题的合并（慎用）

In [ ]:
# topics_to_merge = [1,4]
# model.merge_topics(data['sum'], topics_to_merge)
# headline_topics = model.topics_ # 合并主题后更新各文档的主题
# prob = model.probabilities_ # 合并主题后更新主题概率

## 柱状图可视化

In [ ]:
# 柱状图可视化
fig_bar = model.visualize_barchart(
                         top_n_topics=16, # 前多少个主题
                         n_words=8, # 每个主题展示多少个词汇
                         title='主题词得分', # 若报错，将此行注视掉
                         width=250,height=300 # 图片大小,可调，下面的代码里，其他可视化的函数中这两个参数也可调
                         )

In [ ]:
fig_bar

In [ ]:
from plotly.io import write_html
with open("主题词得分.html", "w", encoding="utf8") as file:
    write_html(fig_bar, file)

## 输出主题词和得分为表格

In [ ]:
# 输出主题词和得分为表格
all_topics = model.get_topics()
topic_df = pd.DataFrame(all_topics).T

topic_words = topic_df[1:len(freq)]
topic_words.to_excel('topic_words.xlsx')

主题关系图

In [ ]:
fig_clu = model.visualize_topics()
fig_clu

In [ ]:
with open("主题关系.html", "w", encoding="utf8") as file:
    write_html(fig_clu, file)

## 层次聚类的可视化

In [ ]:
fig_hierarchy = model.visualize_hierarchy(top_n_topics=16,
                          title='层次聚类图',
                          width=600,
                          height=600)

In [ ]:
fig_hierarchy

In [ ]:
with open("层次聚类图.html", "w", encoding="utf8") as file:
    write_html(fig_hierarchy, file)

## 文档主题聚类图

In [ ]:
fig_doc_topic = model.visualize_documents(
                          topics=list(range(0,16)),
                          docs=summary,
                          hide_document_hover=False,
                          title='文本主题聚类图',# 不同的文本按主题聚类
                          width=1200,
                          height=750
                          )

In [ ]:
fig_doc_topic
# 聚类运行时间较久，请根据需要选用

In [ ]:
with open("文档主题聚类.html", "w", encoding="utf8") as file:
    write_html(fig_doc_topic, file)

## **DTM**
基于bertopic的动态主题模型，注意时间格式需全部都正确才行

In [ ]:
timepoint = data['PubTime-发表时间'].tolist()
timepoint = pd.to_datetime(timepoint, format='%Y%m%d', errors='ignore')
topics_over_time = model.topics_over_time(summary,  # 由摘要生成的变量
                                          timepoint,  # 发表时间生成的变量
                                          datetime_format='mixed', # 如果不能正确识别，先将此行注释掉再重试，时间格式识别在py中不稳定
                                          nr_bins=20,
                                          evolution_tuning=True)

In [ ]:
fig_DTM = model.visualize_topics_over_time(topics_over_time,
                                 top_n_topics=7,
                                 title='DTM',
                                 width=800,
                                 height=350)

In [ ]:
fig_DTM

In [ ]:
with open("DTM图.html", "w", encoding="utf8") as file:
    write_html(fig_DTM, file)

## **主题相似度热力图**

In [ ]:
fig_heatmap = model.visualize_heatmap(top_n_topics=13,
                                      title='主题相似度热力图',
                                      width=800,
                                      height=600)
fig_heatmap

In [ ]:
with open("主题相似度热力图.html", "w", encoding="utf8") as file:
    write_html(fig_heatmap, file)

## **某篇文档的主题概率**

In [ ]:
model.visualize_distribution(model.probabilities_[1], min_probability=0.015)

## **文档主题预测**

可以先减少离群文档数量再进行输出，也可以直接输出

In [ ]:
# # 减少离群文档数量
# new_topics = model.reduce_outliers(summary, headline_topics, strategy="embeddings")
# model.update_topics(summary, topics=new_topics,
#                     vectorizer_model=CountVectorizer(
#                                                     stop_words=stop_words_sklearn,
#                                                     ngram_range=(1,1),
#                                                     binary=False
#                                                     ),
#                     ctfidf_model=ClassTfidfTransformer(
#                                                     bm25_weighting=True,#可选参数，BM25是一种优化的TFIDF类算法，对TFIDF的公式略有修改
#                                                     reduce_frequent_words=True)
#                     )
# # 重新获取主题及主题词列表
# model.get_topic_info()
# # 查看某个主题排名前10的主题词
# model.get_topic(0)

In [ ]:
# Get the topic predictions
topic_prediction = model.topics_[:]
# Save the predictions in the dataframe
data['主题预测'] = topic_prediction
# Take a look at the data
data.head()

In [ ]:
data.to_excel("文档主题预测.xlsx",index=True)